In [1]:
# to mount my drive for simplicity
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import nltk
import string
from nltk.stem import PorterStemmer

w2v = {}
df = pd.read_csv('drive/MyDrive/dataset_lyrics.csv')

nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english'))
docs_without_punctuation = []
ps = PorterStemmer()

docs_without_stopwords = df['lyrics'].apply(lambda x: ' '.join([w for w in x.split() if w not in (stop_words)]))
for row in docs_without_stopwords:
  docs_without_punctuation.append(row.translate(str.maketrans('','', string.punctuation)))
stemmed = [[ps.stem(w) for w in row.split(" ")] for row in docs_without_punctuation]
print(len(stemmed))



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


28372


In [3]:
from gensim.models import Word2Vec
model1 = Word2Vec(stemmed, min_count = 1,size = 10, window = 5, sg=0)
model1.wv.vocab

model2 = Word2Vec(stemmed, min_count = 1,size = 30, window = 10, sg=0)

In [4]:
print(model1.wv['fuck']) # so bekommt man vec zu einem word
print(model2.wv['fuck']) # so bekommt man vec zu einem word

[ 2.6978507  0.6011169  1.0899177 -2.4949007 -4.8949814  3.3804126
 -1.2536362  4.577933  -1.0239915 -1.3083978]
[-1.6111422  -1.9221584   2.2000453  -1.6956457  -0.5824077   4.3288693
  1.1645372   5.231692   -0.8340338  -1.7155808  -3.5833502  -2.8067274
 -0.5069359  -7.351424   -0.38364807  0.3227411   1.0373805  -1.009262
 -1.7844441   1.034159    1.1466757  -3.9346197  -1.4209826  -0.01786908
 -3.5983799   2.7126763  -1.9781492   0.12815912  2.6291695   1.1159222 ]


In [5]:
import numpy as np

# avgerage pooling implementation
def avg_pool(doc, model):
    pool = np.array([0.0]*model.vector_size)
    
    for idx, word in enumerate(doc):
      vec = model.wv[word]
      pool += vec[0]

    pool = pool / len(doc)

        
    return pool

# maximum pooling implementation
def max_pool(doc, model):
    pool = np.array([0.0]*model.vector_size)
    
    for idx, word in enumerate(doc):
      vec = model.wv[word]
      for i, val in np.ndenumerate(vec):
        if abs(val) > abs(pool[i[1]]):
          pool[i[1]] = val
        
    return pool

# avgerage pool list of wvs
def avg_pool_wvs(wvs):
    pool = np.array([0.0]*len(wvs[0]))
    
    for idx, song in enumerate(wvs):
      pool += song

    pool = pool / len(wvs)
        
    return pool

# maxiumu pool list of wvs
def max_pool_wvs(wvs):
    pool = np.array([0.0]*len(wvs[0]))
    
    for idx, song in enumerate(wvs):
      for i, val in np.ndenumerate(song):
        if abs(val) > abs(pool[i[0]]):
          pool[i[0]] = val
        
    return pool

# cosine similarity implementation
def cosine_sim(a, b):
  dot = np.dot(a, b)
  aNorm = np.linalg.norm(a)
  bNorm = np.linalg.norm(b)

  denom = (aNorm * bNorm)

  if denom > 0.0:
    return dot / denom
  elif dot == 0.0:
    return 0.0

In [6]:
track_to_lyrics = dict()

track_to_wv = dict()
artist_to_wv = dict()
artist_to_wvs = dict()
genre_to_wv = dict()
genre_to_wvs = dict()
track_to_wv_test = dict()

track_to_wv_max = dict()
artist_to_wv_max = dict()
artist_to_wvs_max = dict()
genre_to_wv_max = dict()
genre_to_wvs_max = dict()
track_to_wv_test_max = dict()

def reset_globals():
  # reset globals 
  global track_to_lyrics

  global track_to_wv
  global artist_to_wv
  global artist_to_wvs
  global genre_to_wv
  global genre_to_wvs
  global track_to_wv_test

  global track_to_wv_max
  global artist_to_wv_max
  global artist_to_wvs_max
  global genre_to_wv_max
  global genre_to_wvs_max
  global track_to_wv_test_max
  
  track_to_lyrics = dict()

  track_to_wv = dict()
  artist_to_wv = dict()
  artist_to_wvs = dict()
  genre_to_wv = dict()
  genre_to_wvs = dict()
  track_to_wv_test = dict()

  track_to_wv_max = dict()
  artist_to_wv_max = dict()
  artist_to_wvs_max = dict()
  genre_to_wv_max = dict()
  genre_to_wvs_max = dict()
  track_to_wv_test_max = dict()

def extract_data(model, test_size):
  reset_globals()

  test = test_size
  cnt = 0

  # split dataset into test, train data
  for index, row in df.iterrows():
    # prepare train data
    if cnt < test:
      docs_without_punctuation = []
      docs_without_stopwords = [w for w in row['lyrics'].split() if w not in (stop_words)]
      docs_without_punctuation = [w for w in docs_without_stopwords if w not in (string.punctuation)]

      stemmed = [[ps.stem(w) for w in roww.split(" ")] for roww in docs_without_punctuation]

      track_to_lyrics[str(row['artist_name']) + " " + str(row['track_name']) + " " + str(row['release_date'])] = stemmed
      track_to_wv[str(row['artist_name']) + " " + str(row['track_name']) + " " + str(row['release_date'])] = avg_pool(stemmed, model)
      track_to_wv_max[str(row['artist_name']) + " " + str(row['track_name']) + " " + str(row['release_date'])] = max_pool(stemmed, model)

      if str(row['artist_name']) not in artist_to_wvs:
        artist_to_wvs[str(row['artist_name'])] = []
        artist_to_wvs_max[str(row['artist_name'])] = []
      if str(row['genre']) not in genre_to_wvs:
        genre_to_wvs[str(row['genre'])] = []
        genre_to_wvs_max[str(row['genre'])] = []
      artist_to_wvs[str(row['artist_name'])].append(avg_pool(stemmed, model))
      artist_to_wvs_max[str(row['artist_name'])].append(max_pool(stemmed, model))
      genre_to_wvs[str(row['genre'])].append(avg_pool(stemmed, model))
      genre_to_wvs_max[str(row['genre'])].append(max_pool(stemmed, model))

    # or prepare test data
    elif cnt >= test:
      docs_without_punctuation = []
      docs_without_stopwords = [w for w in row['lyrics'].split() if w not in (stop_words)]
      docs_without_punctuation = [w for w in docs_without_stopwords if w not in (string.punctuation)]

      stemmed = [[ps.stem(w) for w in roww.split(" ")] for roww in docs_without_punctuation]

      track_to_lyrics[str(row['artist_name']) + " " + str(row['track_name']) + " " + str(row['release_date'])] = stemmed
      track_to_wv_test[str(row['artist_name']) + " " + str(row['track_name']) + " " + str(row['release_date'])] = avg_pool(stemmed, model)
      track_to_wv_test_max[str(row['artist_name']) + " " + str(row['track_name']) + " " + str(row['release_date'])] = max_pool(stemmed, model)
    
    if cnt == int(test*1.2):
      break
    cnt = cnt + 1

  for name, songs in artist_to_wvs.items():
    artist_to_wv[name] = avg_pool_wvs(songs)
    artist_to_wv_max[name] = max_pool_wvs(songs)

  for name, songs in genre_to_wvs.items():
    genre_to_wv[name] = avg_pool_wvs(songs)
    genre_to_wv_max[name] = max_pool_wvs(songs)

extract_data(model1, 10000)

print(len(artist_to_wv))
print(artist_to_wv['ricky skaggs'])
print(artist_to_wv_max['ricky skaggs'])

2047
[-1.25900216  3.03790152  1.86656255 -1.00368087 -1.87904362 -0.84361939
  0.51228449  0.35102078 -1.23294863  0.58253538]
[-2.49165258  4.52996241  3.195276   -1.99602459 -3.93496418 -1.89370839
  3.00361792  1.78722153 -2.15949199  1.54230765]


In [7]:
# remove stopwords, punctuaction, and do stemming.
# returns stemmed query and average pooled word vector
def preprocess_query(user_query, model, pooling='avg'):
  user_query_without_stopwords = []
  user_query_without_punctuation = []
  user_query_without_stopwords = [w for w in user_query.split() if w not in (stop_words)]
  user_query_without_punctuation = [w for w in user_query_without_stopwords if w not in (string.punctuation)]

  user_query_stemmed = [[ps.stem(w) for w in roww.split(" ")] for roww in user_query_without_punctuation]
  if pooling == 'avg':
    return user_query_stemmed, avg_pool(user_query_stemmed, model)
  elif pooling == 'max':
    return user_query_stemmed, max_pool(user_query_stemmed, model)

In [8]:
# the doors - hello, i love you
the_doors_user_query = "hello i love you"

user_query_stemmed, user_query_stemmed_wv = preprocess_query(the_doors_user_query, model1)

In [9]:
import torch
from torch import nn

# Create model
device = "cpu"

# Define model
class LyricsSimilarityModel1(nn.Module):
    def __init__(self):
        super(LyricsSimilarityModel1, self).__init__()
        self.linear = torch.nn.Sequential(
            torch.nn.Linear(10, 10),
            torch.nn.ReLU(),
            torch.nn.Linear(10, 1)
            
        )

    def forward(self, inputs_):
        x = self.linear(inputs_)
        return x

lyricsSimModel1 = LyricsSimilarityModel1().to(device)
lyricsSimModel1 = lyricsSimModel1.float()

In [10]:
def train(data, user_query_stemmed, model, loss_fn, optimizer):
  model.train()

  for key, item in data.items():
    # predict output using one hot encoding of word

    pred = model(torch.tensor(item).float())

    target = cosine_sim(avg_pool(user_query_stemmed, model1), item)

    #print(pred)
    #print(torch.tensor([target]).float())

    # Compute prediction error
    loss = loss_fn(pred, torch.tensor([target]).float())

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [11]:
def test(data, user_query_stemmed, model, loss_fn, optimizer):
  model.eval()

  threshold = 0.01
  correct = 0

  for key, item in data.items():
    # predict output using one hot encoding of word

    pred = model(torch.tensor(item).float())

    target = cosine_sim(avg_pool(user_query_stemmed, model1), item)

    #print(pred)
    #print(torch.tensor([target]).float())

    # Compute prediction error
    loss = loss_fn(pred, torch.tensor([target]).float())
    #print(loss)
    #print()

    if (target.item() - pred.item()) <= threshold:
      correct += 1
  
  print("accuracy: " + str(correct / len(data.items())))
  return

In [12]:
# Baseline LyricsModel1 

optimizer = torch.optim.Adagrad(lyricsSimModel1.parameters(), lr=0.2)
loss_fn = torch.nn.BCEWithLogitsLoss()

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(track_to_wv, user_query_stemmed, lyricsSimModel1, loss_fn, optimizer)
    test(track_to_wv_test, user_query_stemmed, lyricsSimModel1, loss_fn, optimizer)
print("Done!")

Epoch 1
-------------------------------
accuracy: 0.7081459270364817
Epoch 2
-------------------------------
accuracy: 0.7161419290354822
Epoch 3
-------------------------------
accuracy: 0.7136431784107946
Epoch 4
-------------------------------
accuracy: 0.7171414292853573
Epoch 5
-------------------------------
accuracy: 0.7171414292853573
Done!


In [13]:
# LyricsModel1 with other loss / optimizer

lyricsSimModel1 = LyricsSimilarityModel1().to(device)
lyricsSimModel1 = lyricsSimModel1.float()

# use mseloss because this is not a classification task
# also change optimizer for testing
optimizer = torch.optim.SGD(lyricsSimModel1.parameters(), lr=0.05)
loss_fn = torch.nn.MSELoss()

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(track_to_wv, user_query_stemmed, lyricsSimModel1, loss_fn, optimizer)
    test(track_to_wv_test, user_query_stemmed, lyricsSimModel1, loss_fn, optimizer)
print("Done!")

Epoch 1
-------------------------------
accuracy: 0.5727136431784108
Epoch 2
-------------------------------
accuracy: 0.5982008995502249
Epoch 3
-------------------------------
accuracy: 0.5642178910544727
Epoch 4
-------------------------------
accuracy: 0.5667166416791605
Epoch 5
-------------------------------
accuracy: 0.6091954022988506
Done!


In [14]:
# Create model
device = "cpu"

# Define 2nd model with more layers
class LyricsSimilarityModel2(nn.Module):
    def __init__(self):
        super(LyricsSimilarityModel2, self).__init__()
        self.linear = torch.nn.Sequential(
            torch.nn.Linear(10, 20),
            torch.nn.ReLU(),
            torch.nn.Linear(20, 10),
            torch.nn.ReLU(),
            torch.nn.Linear(10, 1)
        )

    def forward(self, inputs_):
        x = self.linear(inputs_)
        return x

lyricsSimModel2 = LyricsSimilarityModel2().to(device)
lyricsSimModel2 = lyricsSimModel2.float()

In [15]:
# use mseloss because this is not a classification task
# also change optimizer for testing
optimizer = torch.optim.SGD(lyricsSimModel2.parameters(), lr=0.05)
loss_fn = torch.nn.MSELoss()

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(track_to_wv, user_query_stemmed, lyricsSimModel2, loss_fn, optimizer)
    test(track_to_wv_test, user_query_stemmed, lyricsSimModel2, loss_fn, optimizer)
print("Done!")

Epoch 1
-------------------------------
accuracy: 0.6591704147926037
Epoch 2
-------------------------------
accuracy: 0.6751624187906047
Epoch 3
-------------------------------
accuracy: 0.6616691654172914
Epoch 4
-------------------------------
accuracy: 0.6931534232883558
Epoch 5
-------------------------------
accuracy: 0.7381309345327336
Done!


In [16]:
# Create model
device = "cpu"

# Define 2nd model with more layers
class LyricsSimilarityModel3(nn.Module):
    def __init__(self):
        super(LyricsSimilarityModel3, self).__init__()
        self.linear = torch.nn.Sequential(
            torch.nn.Linear(10, 20),
            torch.nn.ReLU(),
            torch.nn.Linear(20, 30),
            torch.nn.ReLU(),
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),
            torch.nn.Linear(20, 10),
            torch.nn.ReLU(),
            torch.nn.Linear(10, 1),
        )

    def forward(self, inputs_):
        x = self.linear(inputs_)
        return x

lyricsSimModel3 = LyricsSimilarityModel3().to(device)
lyricsSimModel3 = lyricsSimModel3.float()

In [17]:
# use mseloss because this is not a classification task
# also change optimizer for testing
optimizer = torch.optim.SGD(lyricsSimModel3.parameters(), lr=0.05)
loss_fn = torch.nn.MSELoss()

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(track_to_wv, user_query_stemmed, lyricsSimModel3, loss_fn, optimizer)
    test(track_to_wv_test, user_query_stemmed, lyricsSimModel3, loss_fn, optimizer)
print("Done!")

# LyricsModel3 seems to work fine for now

Epoch 1
-------------------------------
accuracy: 0.6071964017991005
Epoch 2
-------------------------------
accuracy: 0.5827086456771614
Epoch 3
-------------------------------
accuracy: 0.6141929035482259
Epoch 4
-------------------------------
accuracy: 0.6021989005497251
Epoch 5
-------------------------------
accuracy: 0.6816591704147926
Done!


In [18]:
# word2vec model results
query = []
for item in user_query_stemmed:
  query.append(item[0])
print(model1.most_similar(query)[:10])

[('lover', 0.921744704246521), ('miss', 0.9207759499549866), ('ohhh', 0.9201785326004028), ('awhil', 0.9045699834823608), ('goodby', 0.896577775478363), ('darl', 0.8887207508087158), ('anytim', 0.8884928226470947), ('payer', 0.8815550208091736), ('darlin', 0.8726433515548706), ('istil', 0.8672235012054443)]


<ipython-input-18-b9f0300b6152>:5: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  print(model1.most_similar(query)[:10])


In [19]:
# add some more user queries based on song lyrics

# Godsmack - Good day to die
godsmack_user_query = "Today is a good day to die"
user_query_stemmed_godsmack, user_query_stemmed_wv_godsmack = preprocess_query(godsmack_user_query, model1)

# this song is not in the dataset!
# Nirvana - Frances farmer will have her revenge on seattle
nirvana_user_query = "I miss the comfort in being sad"
user_query_stemmed_nirvana, user_query_stemmed_wv_nirvana = preprocess_query(nirvana_user_query, model1)

In [20]:
# put all user queries in a list for simplicity
user_queries = [the_doors_user_query, godsmack_user_query, nirvana_user_query]

In [21]:
optimizer = torch.optim.SGD(lyricsSimModel3.parameters(), lr=0.05)
loss_fn = torch.nn.MSELoss()

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(track_to_wv, user_query_stemmed,lyricsSimModel3, loss_fn, optimizer)
    train(track_to_wv, user_query_stemmed_godsmack,lyricsSimModel3, loss_fn, optimizer)
    test(track_to_wv_test, user_query_stemmed, lyricsSimModel3, loss_fn, optimizer)
    test(track_to_wv_test, user_query_stemmed_godsmack, lyricsSimModel3, loss_fn, optimizer)

    # test one query which is not in the dataset and not trained
    test(track_to_wv_test, user_query_stemmed_nirvana, lyricsSimModel3, loss_fn, optimizer)
print("Done!")

Epoch 1
-------------------------------
accuracy: 0.8000999500249875
accuracy: 0.8085957021489255
accuracy: 0.48575712143928035
Epoch 2
-------------------------------
accuracy: 0.792103948025987
accuracy: 0.7331334332833583
accuracy: 0.47176411794102946
Epoch 3
-------------------------------
accuracy: 0.792103948025987
accuracy: 0.7191404297851074
accuracy: 0.46926536731634183
Epoch 4
-------------------------------
accuracy: 0.7996001999000499
accuracy: 0.823088455772114
accuracy: 0.48225887056471767
Epoch 5
-------------------------------
accuracy: 0.8100949525237381
accuracy: 0.887056471764118
accuracy: 0.49375312343828087
Done!


In [22]:
extract_data(model1, 100000)

# get most similar songs to all given user queries
def get_x_most_similar_songs_to(q, x, model, pooling='avg'):
  q_stemmed, q_stemmed_wv = preprocess_query(q, model, pooling)

  most_sim = dict()
  for name, wv in track_to_wv.items():
    most_sim[name] = cosine_sim(wv, q_stemmed_wv)
  print("The 10 most similar (cosine_sim) songs by lyrics to the user query '" + q + "':")
  return sorted(most_sim.items(), key=lambda item: item[1], reverse=True)[:x]

for q in user_queries:
  print(get_x_most_similar_songs_to(q, 10, model1))

The 10 most similar (cosine_sim) songs by lyrics to the user query 'hello i love you':
[('ray price letters have no arms 1957', 0.9586160470448446), ('dean martin sweet, sweet lovable you 1967', 0.9562377712563521), ('mickey & sylvia forever and a day 1957', 0.956221419221108), ('the meters be my lady 1977', 0.9553070063649277), ("the beach boys i'll bet he's nice 1977", 0.9550304951320499), ('elvis presley lover doll 1958', 0.9548289152123595), ('chuck berry sweet sixteen 1961', 0.9543354397289383), ('the knack heartbeat 1979', 0.9523785258314573), ('tyrese sweet lady 1998', 0.9522987994134279), ('steve miller band i wanna be loved (but by only you) 1986', 0.9506482614172861)]
The 10 most similar (cosine_sim) songs by lyrics to the user query 'Today is a good day to die':
[("merle haggard i'd trade all of my tomorrows 1965", 0.9927285619137188), ('willie nelson yesterday 1966', 0.9896208547021594), ('mishka as you are 2013', 0.9888600973553411), ('the cascades my first day alone 1963'

In [23]:
# get most similar artists to a given artist
def get_x_most_similar_artists_to(artist_name, artist_to_wv, x):
  artist_wv = artist_to_wv[artist_name]
  most_sim = dict()
  for name, wv in artist_to_wv.items():
    most_sim[name] = cosine_sim(wv, artist_wv)
  print("The 10 most similar (cosine_sim) songs by lyrics to the user query '" + artist_name + "':")

  return sorted(most_sim.items(), key=lambda item: item[1], reverse=True)[:x]

# test model1
print(get_x_most_similar_artists_to('snoop dogg', artist_to_wv, 10))
print(get_x_most_similar_artists_to('nirvana', artist_to_wv, 10))
print(get_x_most_similar_artists_to('snoop dogg', artist_to_wv_max, 10))


# test model2
#extract_data(model2)
#print(get_x_most_similar_artists_to('snoop dogg', 10))

The 10 most similar (cosine_sim) songs by lyrics to the user query 'snoop dogg':
[('snoop dogg', 1.0000000000000002), ('royce da 5\'9"', 0.9946459615333343), ('lloyd banks', 0.9945839928345579), ('greyboy', 0.9943370615649383), ('a$ap ferg', 0.9935264943298352), ('young money', 0.9929512799690751), ('big scoob', 0.9926896241417246), ('kool moe dee', 0.9922991337233015), ('yung joc', 0.9921437374606119), ('foxy brown', 0.9918739750010847)]
The 10 most similar (cosine_sim) songs by lyrics to the user query 'nirvana':
[('nirvana', 1.0), ('hazhe', 0.9961693678261242), ('the dillinger escape plan', 0.9958114354258433), ('motörhead', 0.9950674037586098), ('incubus', 0.9949976686482405), ('part one tribe', 0.9932637604081263), ('tool', 0.9929175841556861), ('paul bley', 0.9928523725295632), ('trae tha truth', 0.9926003007585478), ('alice cooper', 0.992134520984506)]
The 10 most similar (cosine_sim) songs by lyrics to the user query 'snoop dogg':
[('snoop dogg', 0.9999999999999998), ('d.ace', 

In [24]:
# get artist which is most likely to write these words
def get_x_most_similar_artists_to(user_input, x, wvs, model, pooling='avg'):
  user_input_stemmed, user_input_wv = preprocess_query(user_input, model, pooling)
  most_sim = dict()
  for name, wv in wvs.items():
    most_sim[name] = cosine_sim(wv, user_input_wv)
  print("The 10 artists which write most similar (cosine_sim) songs to the user query '" + user_input + "':")

  return sorted(most_sim.items(), key=lambda item: item[1], reverse=True)[:x]

# test model1
print(get_x_most_similar_artists_to(the_doors_user_query, 10, artist_to_wv,  model1))
print(get_x_most_similar_artists_to(godsmack_user_query, 10, artist_to_wv, model1))
print(get_x_most_similar_artists_to(nirvana_user_query, 10, artist_to_wv, model1))
print()
print(get_x_most_similar_artists_to(the_doors_user_query, 10, artist_to_wv_max, model1, 'max'))
print(get_x_most_similar_artists_to(godsmack_user_query, 10, artist_to_wv_max, model1, 'max'))
print(get_x_most_similar_artists_to(nirvana_user_query, 10, artist_to_wv_max, model1, 'max'))


# test model2
#extract_data(model2)
#print(get_x_most_similar_artists_to('snoop dogg', 10))

The 10 artists which write most similar (cosine_sim) songs to the user query 'hello i love you':
[('mickey & sylvia', 0.956221419221108), ('harry james', 0.9468267245060928), ('daniel johnston', 0.9315104598521069), ('gerry mulligan', 0.9275648082777135), ('the del-vikings', 0.9247673693788117), ('the academy is...', 0.9231157683643596), ('a.r. kane', 0.9228363070778688), ('the spaniels', 0.9206420656903055), ('stampeders', 0.9173887961071379), ('selena gomez & the scene', 0.9165590817895273)]
The 10 artists which write most similar (cosine_sim) songs to the user query 'Today is a good day to die':
[('the walkmen', 0.9697720139228403), ('brenda boykin', 0.9637588735336712), ('marmalade', 0.9629010843035772), ('vertical horizon', 0.9587318417379899), ('ronnie laws', 0.9541972191945651), ('hanson', 0.9497331282399825), ('sparklehorse', 0.945806259555627), ('paul hardcastle', 0.9431038400189736), ('lindisfarne', 0.9430245325225101), ('eddie "cleanhead" vinson', 0.942319277061298)]
The 10 

In [25]:
# get genre which is most likely to use these words
def get_x_most_similar_genres_to(user_input, x, genre_to_wv, model, pooling='avg'):
  user_input_stemmed, user_input_wv = preprocess_query(user_input, model, pooling)
  most_sim = dict()
  for name, wv in genre_to_wv.items():
    most_sim[name] = cosine_sim(wv, user_input_wv)
  print("The 10 genre which are most likely (cosine_sim) to use the words in the user query '" + user_input + "':")

  return sorted(most_sim.items(), key=lambda item: item[1], reverse=True)[:x]
# test model1
print(get_x_most_similar_genres_to(the_doors_user_query, 10, genre_to_wv,  model1))
print(get_x_most_similar_genres_to(godsmack_user_query, 10, genre_to_wv, model1))
print(get_x_most_similar_genres_to(nirvana_user_query, 10, genre_to_wv, model1))
print()
print(get_x_most_similar_genres_to(the_doors_user_query, 10, genre_to_wv_max, model1, 'max'))
print(get_x_most_similar_genres_to(godsmack_user_query, 10, genre_to_wv_max, model1, 'max'))
print(get_x_most_similar_genres_to(nirvana_user_query, 10, genre_to_wv_max, model1, 'max'))


# test model2
#extract_data(model2)
#print(get_x_most_similar_artists_to('snoop dogg', 10))

The 10 genre which are most likely (cosine_sim) to use the words in the user query 'hello i love you':
[('country', 0.796620031094254), ('jazz', 0.7906652548419123), ('pop', 0.7814905194902175), ('blues', 0.7801952149739402), ('rock', 0.7397571228227112), ('reggae', 0.7303269395260404), ('hip hop', 0.575543838751476)]
The 10 genre which are most likely (cosine_sim) to use the words in the user query 'Today is a good day to die':
[('country', 0.8623599215642681), ('jazz', 0.8614660246067655), ('pop', 0.8526580215315239), ('rock', 0.8467875755868972), ('reggae', 0.8451255200386928), ('blues', 0.8437251254653569), ('hip hop', 0.6888155430817503)]
The 10 genre which are most likely (cosine_sim) to use the words in the user query 'I miss the comfort in being sad':
[('jazz', 0.8882231092162909), ('country', 0.8842466216652388), ('pop', 0.8710113801877711), ('rock', 0.8700757029393915), ('blues', 0.8680081367028393), ('reggae', 0.8222553795604077), ('hip hop', 0.6014058587658485)]

The 10 gen

In [26]:
# get genre which is most likely to use these words
def get_x_most_similar_genres_for_artist(artist_name, x, artist_to_wv, genre_to_wv, model, pooling='avg'):
  artist_wv = artist_to_wv[artist_name]
  most_sim = dict()
  for name, wv in genre_to_wv.items():
    most_sim[name] = cosine_sim(wv, artist_wv)
  print("The 10 genre which are most likely (cosine_sim) to use the words in the user query '" + artist_name + "':")

  return sorted(most_sim.items(), key=lambda item: item[1], reverse=True)[:x]
# test model1
print(get_x_most_similar_genres_for_artist('snoop dogg', 10, artist_to_wv, genre_to_wv,  model1))
print(get_x_most_similar_genres_for_artist('rick astley', 10, artist_to_wv, genre_to_wv,  model1))
print(get_x_most_similar_genres_for_artist('godsmack', 10, artist_to_wv, genre_to_wv,  model1))
print(get_x_most_similar_genres_for_artist('johnny cash', 10, artist_to_wv, genre_to_wv,  model1))
print(get_x_most_similar_genres_for_artist('inner circle', 10, artist_to_wv, genre_to_wv,  model1))
print(get_x_most_similar_genres_for_artist('rihanna', 10, artist_to_wv, genre_to_wv,  model1))


The 10 genre which are most likely (cosine_sim) to use the words in the user query 'snoop dogg':
[('hip hop', 0.97684384330535), ('reggae', 0.833151263442017), ('blues', 0.7727106267864852), ('pop', 0.7686291844400953), ('rock', 0.764032957122834), ('jazz', 0.7500998288354865), ('country', 0.7292297074740192)]
The 10 genre which are most likely (cosine_sim) to use the words in the user query 'rick astley':
[('pop', 0.964625876087315), ('country', 0.9625354053345673), ('blues', 0.9609672323680224), ('jazz', 0.9569091734541025), ('rock', 0.9552846705820421), ('reggae', 0.9351900040484364), ('hip hop', 0.785903491199145)]
The 10 genre which are most likely (cosine_sim) to use the words in the user query 'godsmack':
[('rock', 0.9857445685414441), ('reggae', 0.9781624504836236), ('pop', 0.9780609599860592), ('blues', 0.9723866274473904), ('jazz', 0.9683980434266922), ('country', 0.9672778814008882), ('hip hop', 0.8750324965431838)]
The 10 genre which are most likely (cosine_sim) to use the 

In [27]:
extract_data(model2, 100000)

# test model2

for q in user_queries:
  print(get_x_most_similar_songs_to(q, 10, model2))
print()
print()
print(get_x_most_similar_artists_to('snoop dogg', 10, artist_to_wv, model2))
print(get_x_most_similar_artists_to('nirvana', 10, artist_to_wv, model2))
print(get_x_most_similar_artists_to('snoop dogg', 10, artist_to_wv_max, model2))
print()
print()
print(get_x_most_similar_artists_to(the_doors_user_query, 10, artist_to_wv,  model2))
print(get_x_most_similar_artists_to(godsmack_user_query, 10, artist_to_wv, model2))
print(get_x_most_similar_artists_to(nirvana_user_query, 10, artist_to_wv, model2))
print()
print(get_x_most_similar_artists_to(the_doors_user_query, 10, artist_to_wv_max, model2, 'max'))
print(get_x_most_similar_artists_to(godsmack_user_query, 10, artist_to_wv_max, model2, 'max'))
print(get_x_most_similar_artists_to(nirvana_user_query, 10, artist_to_wv_max, model2, 'max'))
print()
print()
print(get_x_most_similar_genres_to(the_doors_user_query, 10, genre_to_wv,  model2))
print(get_x_most_similar_genres_to(godsmack_user_query, 10, genre_to_wv, model2))
print(get_x_most_similar_genres_to(nirvana_user_query, 10, genre_to_wv, model2))
print()
print(get_x_most_similar_genres_to(the_doors_user_query, 10, genre_to_wv_max, model2, 'max'))
print(get_x_most_similar_genres_to(godsmack_user_query, 10, genre_to_wv_max, model2, 'max'))
print(get_x_most_similar_genres_to(nirvana_user_query, 10, genre_to_wv_max, model2, 'max'))
print()
print()
print(get_x_most_similar_genres_for_artist('snoop dogg', 10, artist_to_wv, genre_to_wv,  model2))
print(get_x_most_similar_genres_for_artist('rick astley', 10, artist_to_wv, genre_to_wv,  model2))
print(get_x_most_similar_genres_for_artist('godsmack', 10, artist_to_wv, genre_to_wv,  model2))
print(get_x_most_similar_genres_for_artist('johnny cash', 10, artist_to_wv, genre_to_wv,  model2))
print(get_x_most_similar_genres_for_artist('inner circle', 10, artist_to_wv, genre_to_wv,  model2))
print(get_x_most_similar_genres_for_artist('rihanna', 10, artist_to_wv, genre_to_wv,  model2))

The 10 most similar (cosine_sim) songs by lyrics to the user query 'hello i love you':
[('chubby checker lovely, lovely (loverly, loverly) 1964', 0.876051846154941), ('colin blunstone she loves the way they love her 1971', 0.8446512522655714), ('michael franks loving you more and more 1980', 0.8350514394081328), ('george strait all of me (loves all of you) 1992', 0.8346475048739093), ("the marshall tucker band i'll be loving you 1978", 0.8345703057383195), ('fred astaire he loves and she loves 1957', 0.833767077366008), ('michael franks i love lucy 1993', 0.833639805321918), ('george shearing sweet and lovely 1953', 0.830268511668252), ("the beach boys i'll bet he's nice 1977", 0.8294646420446858), ("perry como it's a lovely day today 2001", 0.8290701085001748)]
The 10 most similar (cosine_sim) songs by lyrics to the user query 'Today is a good day to die':
[('the fall to nk roachment: yarbles 1985', 0.8791955123107572), ('andy williams may each day 1970', 0.8739061746136237), ("merle 